In [1]:
import os
import pickle
import datetime
import numpy as np
import pandas as pd

In [2]:
cur_dir = os.getcwd()
main_dir = os.path.dirname(os.path.dirname(cur_dir))
data_dir = os.path.join(main_dir, "ConsolidatedData", "Dynamic1_Jan2023")

all_df_dataname = "videotimed_labeled_consolidated_all.pkl"

all_df_pkl = os.path.join(data_dir, all_df_dataname)

with open(all_df_pkl, 'rb') as f:
  all_df = pickle.load(f)

In [3]:
run_all_datetimes = all_df["Date/Time"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d_%H-%M-%S-%f"))
run_timing = dict()

unique_runs = all_df["Run Number"].unique()
run_start_row_eds = 0

for run_number in unique_runs:
  print (f"Processing run: {run_number}")

  run_start_time = run_all_datetimes.iloc[run_start_row_eds]
  run_row_cnt_eds = all_df["Run Number"].value_counts()[run_number]
  if run_start_row_eds + run_row_cnt_eds < run_all_datetimes.shape[0]:
    run_end_time = run_all_datetimes.iloc[run_start_row_eds + run_row_cnt_eds]
  else:
    run_end_time = run_all_datetimes.iloc[-1]
  
  run_timing[run_number] = (run_start_time, run_end_time)

  run_start_row_eds += run_row_cnt_eds


Processing run: 15
Processing run: 16
Processing run: 17
Processing run: 18
Processing run: 19
Processing run: 20
Processing run: 21


In [4]:
groundtruth_dict = dict()
timestep = datetime.timedelta(0,0,100000) #Predicitons in every 333 SG/PZT samples. In time-scale, we need to make predictions in every 333/10000 second.

row_ix = 1
start_row_ix = row_ix

for run_number in unique_runs:
  print (f"Processing run {run_number}")
  groundtruth_df = pd.DataFrame(columns=["Video Time (sec)", 
                                         "WT_Airspeed", "WT_Airspeed_int", 
                                         "WT_AoA", "WT_AoA_int", 
                                         "UAV_TAS", "UAV_TAS_int", 
                                         "UAV_IAS", "UAV_IAS_int", 
                                         "UAV_Pitch", "UAV_Pitch_int", 
                                         "UAV_Alpha", "UAV_Alpha_int"])
  (run_start_time, run_end_time) = run_timing[run_number]
  t = run_start_time
  
  while t < run_end_time:
    while run_all_datetimes.iloc[row_ix] <= t+timestep:
      row_ix += 1
    vid_time = all_df["Video Time (sec)"].iloc[start_row_ix:row_ix].mean()
    
    wt_airspeed = all_df["MPH"].iloc[start_row_ix:row_ix].mean() * 0.44704
    wt_airspeed_int = np.rint(wt_airspeed)

    wt_aoa = all_df["Inclination (deg)"].iloc[start_row_ix:row_ix].mean()
    wt_aoa_int = np.rint(wt_aoa)

    uav_tas = all_df["TAS"].iloc[start_row_ix:row_ix].mean()
    uav_tas_int = np.rint(uav_tas)

    uav_ias = all_df["IAS"].iloc[start_row_ix:row_ix].mean()
    uav_ias_int = np.rint(uav_ias)

    uav_pitch = all_df["Pitch (deg)"].iloc[start_row_ix:row_ix].mean()
    uav_pitch_int = np.rint(uav_pitch)

    uav_alpha = all_df["ALPHA"].iloc[start_row_ix:row_ix].mean()
    uav_alpha_int = np.rint(uav_alpha)

    groundtruth_df.loc[len(groundtruth_df.index)] = [vid_time, 
                                                     wt_airspeed, wt_airspeed_int, 
                                                     wt_aoa, wt_aoa_int, 
                                                     uav_tas, uav_tas_int, 
                                                     uav_ias, uav_ias_int,
                                                     uav_pitch, uav_pitch_int,
                                                     uav_alpha, uav_alpha_int]

    start_row_ix = row_ix
    t += timestep
  
  groundtruth_dict[run_number] = groundtruth_df


Processing run 15
Processing run 16
Processing run 17
Processing run 18
Processing run 19
Processing run 20
Processing run 21


IndexError: single positional indexer is out-of-bounds

In [ ]:
#Caution: Last DF can be problematic
output_folder = os.path.join(main_dir, "KerasML", "Dynamic1_Jan2023", "labels")

for run_number, example in groundtruth_dict.items():
  example.to_pickle(os.path.join(output_folder,f'dynamic1_run{run_number}_truth_1000sample.pkl'))
  example.to_csv(os.path.join(output_folder,f'dynamic1_run{run_number}_truth_1000sample.csv'), index=False)

In [4]:
val_labels_dir = os.path.join(main_dir, "KerasML", "Dynamic1_Jan2023", "labels")

with open(os.path.join(val_labels_dir,'dynamic1_run15_truth.pkl'), 'rb') as f:
  val_labels = pickle.load(f)